In [7]:
import psutil
import os
import time
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

process = psutil.Process(os.getpid())

def print_ram():
    mem = process.memory_info().rss / (1024 ** 2)
    print(f"RAM usage: {mem:.2f} MB")

# Lokaler Pfad zu deinem gefinetunten Modell-Checkpoint (das Output-Verzeichnis vom Training)
MODEL_PATH = "qwen3-finetune/checkpoint-310"  # ggf. anpassen

# Prompt (gleicher Stil wie beim Finetuning!)
input_text = "Drive 10 meters forward pwease"
system_prompt = "<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n"
user_prompt = f"<|im_start|>user\n{input_text}<|im_end|>\n"
full_prompt = system_prompt + user_prompt + "<|im_start|>assistant\n"

print_ram()
# Model und Tokenizer laden
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH, trust_remote_code=True)

#quantization_config = BitsAndBytesConfig(
#    load_in_4bit=True,                   # oder load_in_8bit=True
#    bnb_4bit_use_double_quant=True,
#    bnb_4bit_quant_type="nf4",           # oft: "nf4" oder "fp4"
#    bnb_4bit_compute_dtype="float16",    # float16 ist Standard
#)

quantization_config = BitsAndBytesConfig(
    load_in_8bit=True
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    device_map="auto",
    #quantization_config=quantization_config,
    trust_remote_code=True,
)

# Tokenisieren
inputs = tokenizer(full_prompt, return_tensors="pt").to(model.device)

# Generieren
output = model.generate(
    **inputs,
    max_new_tokens=128,            # wieviel Text soll generiert werden?
    temperature=0.7,               # Kreativität/Randomness
    do_sample=True,                # Sampling statt Greedy
    top_p=0.9,                     # Nucleus Sampling
    eos_token_id=tokenizer.eos_token_id
)

print_ram()
# Antwort dekodieren (nur den Assistant-Part extrahieren)
decoded = tokenizer.decode(output[0], skip_special_tokens=False)

# Nur den generierten Text NACH dem "<|im_start|>assistant\n" anzeigen:
if "<|im_start|>assistant\n" in decoded:
    answer = decoded.split("<|im_start|>assistant\n", 1)[1]
    # Optional: alles hinter einem evtl. "<|im_end|>" abschneiden
    answer = answer.split("<|im_end|>", 1)[0].strip()
else:
    answer = decoded

print_ram()
print("Antwort:", answer)

RAM usage: 356.55 MB
RAM usage: 418.12 MB
RAM usage: 418.12 MB
Antwort: MOVE FORWARD 10;


In [9]:
while True:
    user_input = input("User: ")
    prompt = (
        "<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n"
        f"<|im_start|>user\n{user_input}<|im_end|>\n"
        "<|im_start|>assistant\n"
    )
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    output = model.generate(**inputs, max_new_tokens=256, temperature=0.7, do_sample=True, top_p=0.9)
    decoded = tokenizer.decode(output[0], skip_special_tokens=False)
    if "<|im_start|>assistant\n" in decoded:
        answer = decoded.split("<|im_start|>assistant\n", 1)[1]
        answer = answer.split("<|im_end|>", 1)[0].strip()
    else:
        answer = decoded
    print("Assistant:", answer)

Assistant: COMMAND NOT RECOGNIZED;
Assistant: COMMAND NOT RECOGNIZED;
Assistant: COMMAND NOT RECOGNIZED;


KeyboardInterrupt: Interrupted by user